In [1]:
import database_utils
import pandas as pd

pd.set_option('display.max_columns', None)

myObject = database_utils.DatabaseConnector()
engine = myObject.init_db_engine()

myObject.list_db_tables(engine)


['legacy_store_details', 'legacy_users', 'orders_table']

In [2]:
df = myObject.read_rds_table(engine,"orders_table")

type(df)

pandas.core.frame.DataFrame

df is a dataframe that now contains the data from our orders table
These are the first & last few lines of the database then!

In [3]:
df.head(5)

,level_0,index,date_uuid,first_name,last_name,user_uuid,card_number,store_code,product_code,1,product_quantity
0,0,0,9476f17e-5d6a-4117-874d-9cdb38ca1fa6,None,None,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8,30060773296197,BL-8387506C,R7-3126933h,NaN,3
1,1,1,0423a395-a04d-4e4a-bd0f-d237cbd5a295,None,None,8fe96c3a-d62d-4eb5-b313-cf12d9126a49,349624180933183,WEB-1388012W,C2-7287916l,NaN,2
2,2,2,65187294-bb16-4519-adc0-787bbe423970,None,None,fc461df4-b919-48b2-909e-55c95a03fe6b,3529023891650490,CH-01D85C8D,S7-1175877v,NaN,2
3,3,3,579e21f7-13cb-436b-83ad-33687a4eb337,None,None,6104719f-ef14-4b09-bf04-fb0c4620acb0,213142929492281,CL-C183BE4B,D8-8421505n,NaN,2
4,4,4,00ab86c3-2039-4674-b9c1-adbcbbf525bd,None,None,9523a6d3-b2dd-4670-a51a-36aebc89f579,502067329974,SO-B5B9CB3B,B6-2596063a,NaN,2


In [4]:
df.tail(5)

,level_0,index,date_uuid,first_name,last_name,user_uuid,card_number,store_code,product_code,1,product_quantity
120118,110549,110548,f0e8fff6-9998-4661-954b-0e258e09d33c,None,None,95c74b0a-d495-4359-b1c0-e2da511e8403,575421945446,KA-FA7ED3B8,C9-6827622o,NaN,4
120119,82164,82164,1c80940a-d186-4ba9-9daa-8abd1aceae32,None,None,5d6fa6fe-e583-4baf-8bbb-d1dd6e2b551f,4971858637664481,WA-A41DA979,I0-1146408B,NaN,1
120120,97599,97599,58598aca-049c-418e-8e39-46327634a7f1,Sharon,Miller,48b7f1fc-db13-4611-ad8e-3dac0b759488,4971858637664481,WEB-1388012W,A4-5443400b,NaN,4
120121,106591,106591,3a76f661-0707-4fbc-9862-f21d3249f581,None,None,51c0b538-7ded-4697-8e84-9f7aa13f9112,4971858637664481,SO-6D328417,E9-2782979e,NaN,4
120122,118806,118804,98fdc8b3-f619-4471-86b4-f44c26880166,None,None,e74907ca-1a4a-476c-a3ca-6b898b0964c2,4971858637664481,SO-6D328417,m0-0281207X,NaN,3


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120123 entries, 0 to 120122
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   level_0           120123 non-null  int64  
 1   index             120123 non-null  int64  
 2   date_uuid         120123 non-null  object 
 3   first_name        15284 non-null   object 
 4   last_name         15284 non-null   object 
 5   user_uuid         120123 non-null  object 
 6   card_number       120123 non-null  int64  
 7   store_code        120123 non-null  object 
 8   product_code      120123 non-null  object 
 9   1                 0 non-null       float64
 10  product_quantity  120123 non-null  int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 10.1+ MB


Well, that's a lot of junk columns! Let's drop: level_0, index, 1, first_name & last_name

In [6]:
df = df.drop(columns=["level_0", "index",  "1",'first_name','last_name'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120123 entries, 0 to 120122
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   date_uuid         120123 non-null  object
 1   user_uuid         120123 non-null  object
 2   card_number       120123 non-null  int64 
 3   store_code        120123 non-null  object
 4   product_code      120123 non-null  object
 5   product_quantity  120123 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 5.5+ MB


Though large, store code and product code should be categories and tied to the other tables

In [7]:
df['store_code'] = df['store_code'].astype('category')
df['product_code'] = df['product_code'].astype('category')

Nope. What about duplicate lines?

In [8]:
df.duplicated().sum()

0

Should I do something about this date_uuid??

Now put the code into a local database maybe

In [9]:
myObject.upload_to_db(df,"orders_table")

Created 123 lines.


Ideas for future:
Misspellings: https://medium.com/geekculture/how-to-fix-misspelled-words-for-your-next-nlp-project-with-one-line-of-code-98888a5ff207
Common variants of companies - "ltd." vs "ltd" vs "limited"